In [43]:
import os
import openai
from abc import ABC, abstractmethod
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# TODO: Give account number as digits instead of the full number
# TODO: Prompt engineering to be as direct as possible
# TODO: It should be a lot more direct

class ContextManager():
    def __init__(self):
        self.context = []
        
    def load_from_file(self, file_path):
        with open(file_path, 'r') as file_handler:
            text = file_handler.read()
            self.context.append(text)
            
    def load_from_directory(self, directory):
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path) and file_path.endswith('.txt'):
                self.load_from_file(file_path)
    
    def load_from_database(self):
        pass
    
    def generate_questions_from_task(self, task, recipient, model="gpt-3.5-turbo"):
        prompt = f"I want to {task} with {recipient}. What questions would they most likely ask me? Can you format your response such that there's one question on each line and no commentary?"
        
        messages = [{"role": "user", "content": prompt}]
        completion = openai.ChatCompletion.create(
            model=model,
            messages=messages
        )
        questions = completion.choices[0].message.content.split("\n")
        questions = [q.strip() for q in questions]
        
        for q in questions:
            self.context.append((q, input(f"Enter the answer to the question - {q}: ")))
                                
        return questions
    
# TODO: Chat engine should keep track of both the context and the dialogue
# TODO: The dialogue is important for the classification task

# class OpenAIChatEngine(ABC):
#     def __init__(self, model, request_limit=1):
#         self.model = model
#         self.requests_till_error = request_limit
        
#     def set_agent_description(self, agent_description=""):
#         if len(agent_description) > 0:
#             return [{"role": "system", "content": agent_description}]
        
#     def __call__(self, messages, counter=0):        
#         try:
#             completion = openai.ChatCompletion.create(
#                 model=self.model,
#                 messages=messages
#             )
#             return completion.choices[0].message
#         except:
#             if counter < self.requests_till_error:
#                 return self.__call__(messages, counter+1)
#             else:
#                 return "I'm sorry, I'm having trouble understanding you. Could you rephrase your request?"

class Agent():
    def __init__(self, task, recipient, efficient_messages=True):
        self.task = task
        self.recipient = recipient
        
        # Setting to understand how to set up the messages
        self.efficient_messages = efficient_messages
        
        # Setup context manager's default value
        self.context_manager = None
        
        # Setup chat engine
        self.model = "gpt-3.5-turbo" 
        agent_description_prompt = f"You're imitating a human that is trying to {task} with {recipient}. Imagine you're on a call with their customer service. Sound like a human and use your context to return the appropriate response. You could use filler words like 'um' and 'uh' to sound more human."
        self.agent_description = [{"role": "system", "content": agent_description_prompt}]
        
        # Setup loggers to keep track of conversation and history
        self.messages = [self.agent_description]
        self.dialogue_history = []
        
    def connect_context(self, context_manager : ContextManager):
        self.context_manager = context_manager
        
    def prompt_enhance_with_context(self):
        if self.context_manager:
            context = "Here's information about the human you're imitating, you can use this to help you respond:"
            for c in self.context_manager.context:
                context += f"\n{c}"
            return context
        else:
            return ""
        
    def engineer_prompt(self, customer_service_response):
        context = self.prompt_enhance_with_context()
        
        ending_prompt = "If the call is over and you've resolved your issue, you can return 'bye' to end the call. If you are unsure about a question and need more information, you can return '/user help' to get assistance."
        formatting_prompt = "Pretend that you're speeking on the phone, so if you need to say any numbers, write them as digits with spaces in between. Like 5032 should be 5 0 3 2."
        
        customer_service_prompt = f"Customer Service Agent: {customer_service_response}"
        agent_response_prompt = f"Your Response:"
        
        components = [context, ending_prompt, formatting_prompt, customer_service_prompt, agent_response_prompt]
        return "\n".join(components)
    
    def generate_response(self, customer_service_response):
        new_dialogue_chain = {"role": "user", "content": customer_service_response}
        new_messages_chain = {"role": "user", "content": self.engineer_prompt(customer_service_response)}
        
        self.dialogue_history.append(new_dialogue_chain)
        self.messages.append(new_messages_chain)
        
        if self.efficient_messages:
            messages = [self.agent_description] + self.dialogue_history[:-1] + [new_messages_chain]
        else:
            messages = self.messages
            
        print("messages", messages)
        
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages=messages
        )
        response = completion.choices[0].message

In [39]:
task = "creating a new account"
recipient = "People's Gas"


context_manager = ContextManager()
context_manager.generate_questions_from_task(task, recipient)


In [44]:
agent = Agent(task, recipient)
agent.connect_context(context_manager)
agent.generate_response("Hi! This is People's Gas, how may we help you today?")

messages [[{'role': 'system', 'content': "You're imitating a human that is trying to creating a new account with People's Gas. Imagine you're on a call with their customer service. Sound like a human and use your context to return the appropriate response. You could use filler words like 'um' and 'uh' to sound more human."}], {'role': 'user', 'content': 'Here\'s information about the human you\'re imitating, you can use this to help you respond:\n(\'What is your full name?\', \'Mehmet Deniz Birlikci\')\n(\'What is your residential address?\', \'5032 Forbes Ave, SMC 4568, Pittsburgh, PA, 15289\')\n(\'What is your mailing address (if different from your residential address)?\', "It\'s the same as residential address")\n(\'What is your phone number?\', \'4126084757\')\n(\'What is your email address?\', \'mdbirlikci@gmail.com\')\n(\'What is your desired payment method?\', \'Credit Card - online\')\n(\'What is your social security number or tax identification number?\', \'123-45-678\')\n(\'

InvalidRequestError: [{'role': 'system', 'content': "You're imitating a human that is trying to creating a new account with People's Gas. Imagine you're on a call with their customer service. Sound like a human and use your context to return the appropriate response. You could use filler words like 'um' and 'uh' to sound more human."}] is not of type 'object' - 'messages.0'

In [36]:
import os
import openai
from abc import ABC, abstractmethod
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# TODO: Give account number as digits instead of the full number
# TODO: Prompt engineering to be as direct as possible
# TODO: It should be a lot more direct

class ContextManager():
    def __init__(self):
        self.context = []
        
    def load_from_file(self):
        pass
    
    def load_from_directory(self):
        pass
    
    def load_from_database(self):
        pass
    
    def generate_questions_from_task(self, task, recipient):
        model = ContextEnhancer()
        model.recipient = recipient
        prompt = f"I want to {task} with {recipient}. What questions would they most likely ask me? Can you format your response such that there's one question on each line and no commentary?"
        
        messages = [{"role": "user", "content": prompt}]
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        questions = completion.choices[0].message.content.split("\n")
        questions = [q.strip() for q in questions]
        return questions
    
class OpenAIChatEngine(ABC):
    def __init__(self, model="gpt-3.5-turbo"):
        self.model = model
        self.messages = []
        
    @abstractmethod
    def set_agent_config(self):
        self.messages = []
        self.agent_description = ""
        self.messages.append({"role": "system", "content": self.agent_description})
        
    def __call__(self, prompt, counter=0):        
        try:
            self.messages.append({"role": "user", "content": prompt})
            
            completion = openai.ChatCompletion.create(
                model=self.model,
                messages=self.messages
            )
            return completion.choices[0].message
        except:
            if counter < 3:
                return self.__call__(prompt, counter+1)
            else:
                return "I'm sorry, I'm having trouble understanding you. Could you rephrase your request?"
    
class ContextEnhancer(OpenAIChatEngine):
    def set_agent_config(self):
        self.agent_description = f"""
            Put yourself in the shoes of a customer service agent for {self.recipient}. 
            Your goal is to be able to figure out what information you need from the customer to finish the task.
        """
        self.messages = []
        self.messages.append({"role": "system", "content": self.agent_description})

In [37]:
ContextManager().generate_questions_from_task("start a new subscription", "gold GYM")

['What is your name?',
 'What is your address?',
 'What is your phone number?',
 'What is your email address?',
 'Are you already a member of Gold Gym?',
 'Which package are you interested in?',
 'When do you plan on starting?',
 'How will you be paying for the subscription?',
 'Do you have any medical conditions that might affect your workout routine?']

# Test whisper

In [16]:
import openai
import os
from dotenv import load_dotenv
import requests
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

audio_url_example = "https://s3.us-west-1.wasabisys.com/blueberryai-input/output_0.mp3"

def convert_speech_to_text_whisper(recording_url):
    # Download the audio file from the URL
    response = requests.get(recording_url)
    audio_file = response.content

    # Save the audio data to a file
    with open("temp.wav", "wb") as file:
        file.write(audio_file)

    # Transcribe the audio using Whisper API
    with open("temp.wav", "rb") as file:
        transcript = openai.Audio.transcribe("whisper-1", file)
    
    return transcript.text

convert_speech_to_text_whisper(audio_url_example)

"Hey, this is People's Gas Company. How can I help you?"

In [ ]:
import json
def string_to_json(string):
    return json.loads(string)
    
def run_hume_transcription(recording_url):
    url = "https://api.hume.ai/v0/batch/jobs"

    payload = "{\"models\":{\"face\":{\"fps_pred\":3,\"prob_threshold\":0.99,\"identify_faces\":false,\"min_face_size\":60,\"save_faces\":false},\"prosody\":{\"granularity\":\"utterance\",\"identify_speakers\":false,\"window\":{\"length\":4,\"step\":1}},\"language\":{\"granularity\":\"word\",\"identify_speakers\":false},\"ner\":{\"identify_speakers\":false}},\"transcription\":{\"language\":null}," + "\"urls\":[\"" + recording_url + "\"],\"notify\":false}"
    headers = {
        "accept": "application/json; charset=utf-8",
        "content-type": "application/json; charset=utf-8",
        "X-Hume-Api-Key": os.getenv("HUME_API_KEY"),
    }
    response = requests.post(url, data=payload, headers=headers)
    job_id = string_to_json(response.text)['job_id']
    
    return job_id

def get_hume_result(job_id):
    while True:
        url = f"https://api.hume.ai/v0/batch/jobs/{job_id}/predictions"

        headers = {
            "accept": "application/json; charset=utf-8",
            "X-Hume-Api-Key": os.getenv("HUME_API_KEY"),
        }

        response = requests.get(url, headers=headers)
        print(string_to_json(response.text))

job_id = run_hume_transcription(audio_url_example)


In [1]:
import requests
import time
import openai
import os
from dotenv import load_dotenv

# Load the variables from the .env file
load_dotenv()

# Set up Twilio client
account_sid = os.getenv('ACCOUNT_SID')
auth_token = os.getenv('AUTH_TOKEN')
twilio_phone_number = os.getenv("TWILIO_PHONE_NUMBER")
recipient_phone_number = os.getenv('RECIPIENT_PHONE_NUMBER')
hume_api_key = os.getenv('HUME_API_KEY')
fliki_api_key = os.getenv('FLIKI_API_KEY')

In [9]:
hume_api_key

'BRlltt9gz2pChEvFQLC38mvEe8jGOoAHuZd4lKeY9vZqydZH'

In [15]:
url = "https://api.fliki.ai/v1/languages"
headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
response.json()

{'success': True,
 'data': [{'_id': '61b8b2f24268666c126babb1', 'name': 'Afrikaans'},
  {'_id': '62b2110fb751a736405225ce', 'name': 'Albanian'},
  {'_id': '61b8b2f24268666c126babb3', 'name': 'Amharic'},
  {'_id': '61b8b2f24268666c126babb5', 'name': 'Arabic'},
  {'_id': '635bc419d0900a036c2e439d', 'name': 'Armenian'},
  {'_id': '62b21780d63b5531a40753f4', 'name': 'Azerbaijani'},
  {'_id': '61b8b2f24268666c126babb7', 'name': 'Bangla/Bengali'},
  {'_id': '635bc666d0900a036c2e6ed2', 'name': 'Basque'},
  {'_id': '62b2188ed63b5531a40757e3', 'name': 'Bosnian'},
  {'_id': '61b8b2f34268666c126babb9', 'name': 'Bulgarian'},
  {'_id': '61b8b2f34268666c126babbb', 'name': 'Burmese'},
  {'_id': '61b8b2f34268666c126babbd', 'name': 'Catalan'},
  {'_id': '61b8b2f44268666c126babbf', 'name': 'Chinese'},
  {'_id': '61b8b2f44268666c126babc1', 'name': 'Croatian'},
  {'_id': '61b8b2f44268666c126babc3', 'name': 'Czech'},
  {'_id': '61b8b2f44268666c126babc5', 'name': 'Danish'},
  {'_id': '61b8b2f54268666c126bab

# Get all the voices

In [ ]:
url = "https://api.fliki.ai/v1/voices"
english_language_id = "61b8b2f54268666c126babc9"
us_dialect_id = "61b8b31c4268666c126bace7"

headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}
data = {
    "languageId": english_language_id,
    "dialectId": us_dialect_id
}

response = requests.post(url, headers=headers, json=data)
response.json()

# Conversational Call

In [27]:
conversational_style_id = "6434632c9f50eacb088edafd"
marcus_speaker_id = "643463179f50eacb088edaec"

import requests

content_text = "Um.. yes I'd like to cancel my subscription."

url = "https://api.fliki.ai/v1/generate/text-to-speech"
headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}
data = {
    "content": content_text,
    "voiceId": marcus_speaker_id,
    "voiceStyleId": conversational_style_id
}

response = requests.post(url, headers=headers, json=data)

import json

response_text = b'{"success":true,"data":{"audio":"https://storage.googleapis.com/fliki/media/api/648e29e780c5a1088ca021f4/648e335155f0dd4c6d681357.mp3","duration":1.968}}'

# Check the response status code
if response.status_code == 200:
    # Process the response
    audio_data = response.content
    # Do something with the audio data
    response_dict = json.loads(audio_data)

    # Now you can access the dictionary elements
    success = response_dict["success"]
    audio_url = response_dict["data"]["audio"]
    duration = response_dict["data"]["duration"]
    
    print(success, audio_url, duration)
else:
    # Handle the error
    print(f"Request failed with status code {response.status_code}: {response.text}")


True https://storage.googleapis.com/fliki/media/api/648e29e780c5a1088ca021f4/648e33a755f0dd4c6d688d55.mp3 2.016
